# Momentum Analysis

How will the past deliveries affect the outcome of the next ball? 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../Data/selected_data/merged_data.csv')
data.info()

C:\Users\kaush\AppData\Local\Temp\ipykernel_34672\1736518648.py:6: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/selected_data/merged_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           217242 non-null  int64  
 1   batter               217242 non-null  object 
 2   bowler               217242 non-null  object 
 3   non_striker          217242 non-null  object 
 4   runs_by_bat          217242 non-null  int64  
 5   extra_runs           217242 non-null  int64  
 6   total_runs_delivery  217242 non-null  int64  
 7   current_team_total   217242 non-null  int64  
 8   runs_remain          103118 non-null  float64
 9   batter_runs          217242 non-null  int64  
 10  balls_faced          217242 non-null  int64  
 11  wickets_fallen       217242 non-null  int64  
 12  delivery             217242 non-null  int64  
 13  over                 217242 non-null  int64  
 14  extra_type           217242 non-null  object 
 15  wicket_type      

In [16]:
momentumData = data[['match_id', 'innings', 'over', 'delivery', 'total_runs_delivery','wicket_type',  'extra_type', 'won']]

# new feature - reball - if the ball is wide or no ball, then the ball is reball. Boolean value
momentumData['reball'] = (momentumData['extra_type'] == 'wides') | (momentumData['extra_type'] == 'noballs')

# delete the extra_type column
momentumData = momentumData.drop('extra_type', axis=1)

# if reball is true, 
# then the delivery of the next ball should be decremented by 1

momentumData['reball'] = momentumData['reball'].astype(int)
momentumData['delivery'] = (momentumData['delivery'] - momentumData['reball'].shift(1).fillna(0)).astype(int)
momentumData['reball'] = momentumData['reball'].astype(bool)	

# new feature - ball - the ball number of the match
momentumData['ball'] = (momentumData['over'] * 6) + momentumData['delivery']

# new feature - isWicket - if the wicket_type is not 0, then the ball is wicket
momentumData['isWicket'] = momentumData['wicket_type'] != '0'

# delete the wicket_type column
momentumData = momentumData.drop('wicket_type', axis=1)

momentumData.head(100)

C:\Users\kaush\AppData\Local\Temp\ipykernel_34672\669283814.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentumData['reball'] = (momentumData['extra_type'] == 'wides') | (momentumData['extra_type'] == 'noballs')


,match_id,innings,over,delivery,total_runs_delivery,won,reball,ball,isWicket
0,Australia_Sri Lanka_2017-02-17,1,0,1,0,0.0,False,1,False
1,Australia_Sri Lanka_2017-02-17,1,0,2,0,0.0,False,2,False
2,Australia_Sri Lanka_2017-02-17,1,0,3,1,0.0,False,3,False
3,Australia_Sri Lanka_2017-02-17,1,0,4,2,0.0,False,4,False
4,Australia_Sri Lanka_2017-02-17,1,0,5,0,0.0,False,5,False
...,...,...,...,...,...,...,...,...,...
95,Australia_Sri Lanka_2017-02-17,1,15,4,1,0.0,False,94,False
96,Australia_Sri Lanka_2017-02-17,1,15,5,2,0.0,False,95,False
97,Australia_Sri Lanka_2017-02-17,1,15,6,1,0.0,True,96,False
98,Australia_Sri Lanka_2017-02-17,1,15,6,6,0.0,False,96,False
